## Библиотеки

In [82]:
import numpy as np
import pandas as pd
import requests

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
#from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVR, SVC
from sklearn.model_selection import GridSearchCV

In [6]:
from sklearn.datasets import make_classification, make_regression

## Данные

In [8]:
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                            random_state=0, shuffle=True)

In [9]:
Z, r = make_regression(n_samples=1000, n_features=10,
                           n_informative=4, random_state=0, shuffle=True)

- Дефолтные параметры
- Подбор параметров
- Предсказание

## Запросы к приложению

In [456]:
r_n = requests.get('http://192.168.1.12:5000/api/ml_models')
r_n

<Response [200]>

In [394]:
#currp = r_n.json()

In [411]:
#currp2 = r_n.json()

In [416]:
currp == currp, currp == currp2

(True, False)

In [459]:
r_n.json()[0]

{'classification': ['Random Forest', 'SVM'],
 'regression': ['Random Forest', 'SVM']}

In [460]:
[s['model_path'] for s in r_n.json()[1]], [s['id'] for s in r_n.json()[1]]

(['models static/classification_SVM_1.pickle',
  'models static/classification_Random Forest_2.pickle'],
 [1, 2])

In [454]:
data={'problem': 'classification', 'name': 'Random Forest', 'h_tune': False, 'X':X.tolist(), 'y':y.tolist()}

In [ ]:
requests.post('http://192.168.1.12:5000/api/ml_models', json=data)

In [452]:
data={'problem': 'classification', 'name': 'SVM', 'h_tune': False, 'X':X.tolist(), 'y':y.tolist()}

In [455]:
requests.post('http://192.168.1.12:5000/api/ml_models', json=data)

<Response [200]>

In [383]:
requests.delete('http://192.168.1.12:5000/api/ml_models/1')

<Response [204]>

In [437]:
requests.put('http://192.168.1.12:5000/api/ml_models/4', 
             json={'problem': 'classification', 'name': 'Random Forest', 'h_tune': True})

<Response [200]>

## Модели

In [71]:
def Classification(model, X, y, htune=False):
    
    if model == 'Random Forest':
        param_grid = {'n_estimators': [50, 100], 'max_depth': [3, 4], 'max_features': ['auto', 'sqrt']}
        clf = RandomForestClassifier(random_state=0)
    elif model == 'SVM':
        param_grid = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
        clf = SVC(random_state=0)
    
    if htune:
        clf_CV = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5)
        clf_CV.fit(X, y)
        #best_params = clf.best_params_
        best_model = clf_CV.best_estimator_
        prediction = best_model.predict(X)
        
    else:
        clf.fit(X, y)
        prediction = clf.predict(X)
        #best_params = clf.get_params()
    
    return prediction #best_params

In [74]:
%%time
prediction = Classification('SVM', X, y, False)
prediction[:10]

CPU times: user 35.5 ms, sys: 2.12 ms, total: 37.6 ms
Wall time: 35.2 ms


array([1, 0, 0, 1, 1, 1, 0, 1, 0, 1])

In [79]:
def Regression(model, X, y, htune=False):
    
    if model == 'Random Forest':
        param_grid = {'n_estimators': [50, 100], 'max_depth': [3, 4], 'max_features': ['auto', 'sqrt']}
        lr = RandomForestRegressor(random_state=0)
    elif model == 'SVM':
        param_grid = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
        lr = SVR()
    
    if htune:
        lr_CV = GridSearchCV(estimator=lr, param_grid=param_grid, cv= 5)
        lr_CV.fit(X, y)
        #best_params = clf.best_params_
        best_model = lr_CV.best_estimator_
        prediction = best_model.predict(X)
        
    else:
        lr.fit(X, y)
        prediction = lr.predict(X)
        #best_params = clf.get_params()
    
    return prediction #best_params

In [81]:
%%time
prediction = Regression('Random Forest', Z, r, False)
prediction[:10]

CPU times: user 446 ms, sys: 8.24 ms, total: 454 ms
Wall time: 454 ms


array([  42.71988408,  -42.73897822,   26.25876795,   48.71589377,
         -4.2270664 ,  -80.92470981,   56.38980455,   46.08721386,
       -140.80379108,   33.87466928])

#### Random Forest

classification

In [24]:
#Random forest GreesSearchCV
rf_param_grid = {'n_estimators': [50, 100],
                 'max_depth': [3, 4],
                 'max_features': ['auto', 'sqrt']
                }

In [25]:
%%time
clf = RandomForestClassifier(random_state=0)
CV_clf = GridSearchCV(estimator=clf, param_grid=rf_param_grid, cv= 5)
CV_clf.fit(X, y)
best_params = CV_clf.best_params_
best_model = CV_clf.best_estimator_
prediction = best_model.predict(X)

CPU times: user 4.55 s, sys: 88.6 ms, total: 4.64 s
Wall time: 4.69 s


regression

In [67]:
%%time
lr = RandomForestRegressor(random_state=0)
CV_lr = GridSearchCV(estimator=lr, param_grid=rf_param_grid, cv= 5)
CV_lr.fit(Z, r)
best_params = CV_lr.best_params_
best_model = CV_lr.best_estimator_
prediction = best_model.predict(Z)

CPU times: user 4.82 s, sys: 34.9 ms, total: 4.85 s
Wall time: 4.85 s


In [68]:
prediction[:10]

array([  43.28182696,  -72.1776449 ,    7.26750592,   21.38554593,
          4.73059485,  -69.06735374,   24.30899387,   39.36748891,
       -136.99463175,   19.87488798])

#### SVM

classification

In [26]:
svm_param_grid = {'kernel':('linear', 'rbf'),
                  'C':[1, 10]
                 }

In [66]:
%%time
clf = SVC(random_state=0)
CV_clf = GridSearchCV(estimator=clf, param_grid=svm_param_grid, cv= 5)
CV_clf.fit(X, y)
best_params = CV_clf.best_params_
best_model = CV_clf.best_estimator_
prediction = best_model.predict(X)

CPU times: user 236 ms, sys: 3.59 ms, total: 239 ms
Wall time: 239 ms


regression

In [69]:
%%time
lr = SVR()
CV_lr = GridSearchCV(estimator=lr, param_grid=svm_param_grid, cv= 5)
CV_lr.fit(Z, r)
best_params = CV_lr.best_params_
best_model = CV_lr.best_estimator_
prediction = best_model.predict(Z)

CPU times: user 755 ms, sys: 5.11 ms, total: 760 ms
Wall time: 760 ms


In [70]:
best_params, best_model, prediction[:10]

({'C': 10, 'kernel': 'linear'},
 SVR(C=10, kernel='linear'),
 array([  55.07942609,  -31.41343535,   37.02373867,   50.39994597,
          -6.60878732,  -83.00622185,   63.80482698,   44.09822181,
        -141.26588642,   34.70977182]))